<a href="https://colab.research.google.com/github/unalf/washingtondc_limedata/blob/main/WashingtonDC_LimeData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Washington DC Dockless Data via API
## Lime Micromobility (including Jump bikes)

DDOT and Dockless Data

As a condition of operating in the District, dockless bikesharing companies are required to provide monthly data reports to DDOT. This data includes the number of rides and vehicles in service, instances of reported safety incidents and improper parking, and aggregated data on repairs. Companies are also required to provide anonymized trip data. This data has helped DDOT evaluate the pilot program and can support current and future planning efforts.

DDOT and the District value open data and data sharing but the data provided in the monthly reports can be sensitive. Given the sensitivity of the data, DDOT is not publicly posting origin-destination data but is seeking other avenues for public access to more information about the performance of the program. DDOT’s goal is to protect the privacy of dockless bikeshare users while maintaining transparency for the residents of the District.

DDOT has worked with several entities to analyze the monthly report data under non-disclosure agreements. The agency is open to collaborating with more research institutions under similar conditions and encourages independent researchers and data scientists to access dockless bikeshare data via the public APIs.

In [ ]:
!pip install --upgrade nbformat
!pip install --upgrade plotly

In [90]:
# Scrape data from https://ddot.dc.gov/page/dockless-api
import json
from urllib.request import Request, urlopen

req = Request('https://data.lime.bike/api/partners/v1/gbfs/washington_dc/free_bike_status.json', headers={'User-Agent': 'Mozilla/5.0'})
data = json.loads(urlopen(req).read().decode())

In [91]:
# Human readable last updated datetime GMT
from datetime import datetime, timezone, timedelta
Washington_Time = datetime.fromtimestamp(data['last_updated'], tz=timezone(timedelta(hours=-8))).strftime('%Y-%m-%d %H:%M:%S')
Local_Time = datetime.fromtimestamp(data['last_updated']).strftime('%Y-%m-%d %H:%M:%S')
print("Washington Time: ", Washington_Time)
print("Server Time: ", Local_Time)

Washington Time:  2021-11-30 05:34:19
Server Time:  2021-11-30 13:34:19


In [92]:
#Convert json to pandas dataframe
import pandas as pd 
df = pd.json_normalize(data['data']['bikes'])
df.head()

,bike_id,lat,lon,is_reserved,is_disabled,vehicle_type,rental_uris.android,rental_uris.ios
0,c870919d-c52b-4564-bb58-83c8cdcadb6d,38.937,-76.9921,0,0,bike,limebike://map?selected_vehicle_id=LG5MOZOISAP...,limebike://map?selected_vehicle_id=LG5MOZOISAP...
1,1fc92d94-7671-44ee-a5e1-178f43b290e0,38.9112,-76.9413,1,0,bike,limebike://map?selected_vehicle_id=A5D4LORPVWQ...,limebike://map?selected_vehicle_id=A5D4LORPVWQ...
2,09380bc0-9ba2-4bcf-bd9f-b0700aba8d9b,38.8649,-76.9511,1,0,bike,limebike://map?selected_vehicle_id=EQVIOBZWDXA...,limebike://map?selected_vehicle_id=EQVIOBZWDXA...
3,dc46c7df-ce3e-4f69-acbd-c0d2cff01396,38.9083,-76.997,0,0,scooter,limebike://map?selected_vehicle_id=C54IFWVOZ2S...,limebike://map?selected_vehicle_id=C54IFWVOZ2S...
4,6b3975a9-ba6c-4fb3-8f75-b09d76febe76,38.9615,-76.9974,0,0,bike,limebike://map?selected_vehicle_id=IKOVQG6UXQA...,limebike://map?selected_vehicle_id=IKOVQG6UXQA...


In [93]:
# Visualize data using map plot
import plotly.express as px

fig = px.scatter_mapbox(df, lat=pd.to_numeric(df['lat']), lon=pd.to_numeric(df['lon']), hover_data=["vehicle_type", "is_reserved"],
                        color='vehicle_type', zoom=11, height=600, title='Washington Time: ' + Washington_Time)
fig.update_layout(mapbox_style="open-street-map")
fig.show()